<a href="https://colab.research.google.com/github/BraedynL0530/Nutrifitness/blob/master/Habit_to_Weight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
import random
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error, r2_score

In [18]:
def generate_weekly_summary(num_weeks=1000):
    """
    Generator that yields synthetic WeeklySummary data.
    Variable names match your Django model fields.
    """
    for _ in range(num_weeks):
        # Basic fields
        sex = random.choice([1, 0])  # 1=male, 0=female

        starting_weight = random.uniform(50, 120)

        # TDEE (already calculated and saved in FitnessProfile)
        tdee = random.uniform(1400, 3500)

        # Calorie intake (varies based on their goal - lose/gain/maintain)
        # Goal already saved, so calories reflect their actual eating
        avg_daily_calories = random.uniform(1200, 4000)

        # Protein intake (grams)
        avg_daily_protein = random.uniform(40, 250)

        # Other macros
        avg_daily_carbs = random.uniform(100, 500)
        avg_daily_fat = random.uniform(30, 150)

        # Calorie deficit/surplus
        avg_calorie_deficit = tdee - avg_daily_calories

        # Consistency score (std deviation)
        consistency_score = random.uniform(50, 400)

        # Weight change calculation
        # ~7700 calories per kg
        expected_change = (avg_calorie_deficit * 7) / 7700
        noise = random.gauss(0, 0.3)
        weight_change = expected_change + noise

        ending_weight = starting_weight + weight_change

        # Dates
        week_start = datetime.now().date() - timedelta(weeks=random.randint(0, 52))
        week_end = week_start + timedelta(days=6)

        yield {
            'week_start': week_start,
            'week_end': week_end,
            'avg_daily_calories': round(avg_daily_calories, 2),
            'avg_daily_protein': round(avg_daily_protein, 2),
            'avg_daily_carbs': round(avg_daily_carbs, 2),
            'avg_daily_fat': round(avg_daily_fat, 2),
            'avg_calorie_deficit': round(avg_calorie_deficit, 2),
            'consistency_score': round(consistency_score, 2),
            'starting_weight': round(starting_weight, 2),
            'ending_weight': round(ending_weight, 2),
            'weight_change': round(weight_change, 2),
            'sex': sex,
            'tdee': round(tdee, 2),
        }

x = []
y = []

for week in generate_weekly_summary(5000):
  features =[
      week['avg_daily_calories'],
      week['avg_daily_protein'],
      week['sex'],
      week['tdee']
  ]
  x.append(features)
  y.append(week['weight_change'])

print(f"{len(x)} samples")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
model = Ridge(alpha=1.0)

model.fit(x_train, y_train)

# Evaluate
y_pred = model.predict(x_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nModel Performance:")
print(f"MAE: {mae:.3f} kg")
print(f"R²: {r2:.3f}")

print(f"\nModel Coefficients:")
feature_names = ['avg_daily_calories', 'avg_daily_protein', 'sex', 'tdee']
for name, coef in zip(feature_names, model.coef_):
    print(f"{name}: {coef:.6f}")
print(f"Intercept: {model.intercept_:.6f}")

5000 samples

Model Performance:
MAE: 0.242 kg
R²: 0.902

Model Coefficients:
avg_daily_calories: -0.000918
avg_daily_protein: -0.000018
sex: -0.008167
tdee: 0.000909
Intercept: 0.031825


In [6]:
# Save model
joblib.dump(model, 'weight_prediction_model.joblib')
print("\nModel saved as 'weight_prediction_model.joblib'")